# Importing Header and SoilPrep

In [21]:
#!pip install tabulate

In [22]:
import import_ipynb 
from HyperMain import *

In [23]:
import import_ipynb 
from FinalHypertunedBestModels import *

In [24]:
legacy_spec = pd.read_csv('spectra.csv')
legacy_tar = pd.read_csv('targets.csv')

legacy_spec.rename(columns= {'sample':'id'}, inplace=True)
legacy_tar.rename(columns = {'SOIL': 'id', 'CLAI':'Clay', 'SILT': 'Silt', 'SAND': 'Sand', 'OC': 'TOC', 'CACO3':'CaCO3'}, inplace = True)

In [25]:
legacy_spec = legacy_spec.set_index('id', drop=True)
legacy_tar = legacy_tar.set_index('id', drop=True)

In [26]:
legacy_tar = legacy_tar[['Sand', 'Silt', 'Clay', 'TOC', 'CaCO3']]

In [27]:
# Missing ID's are taken care by setting the index to ID and obtaining inner join.
legacy_fr = pd.merge(legacy_tar, legacy_spec, on = 'id', how = 'inner')
legacy_fr.reset_index(inplace = True)

In [28]:
print('Missing:', legacy_fr.isnull().sum().sum())   
print(legacy_fr.isnull().sum())
legacy_fr.dropna(axis=0, inplace = True)

Missing: 3
id      0
Sand    1
Silt    1
Clay    1
TOC     0
       ..
2496    0
2497    0
2498    0
2499    0
2500    0
Length: 2157, dtype: int64


In [29]:
df = legacy_fr
# ------------- Target Isolation ----------------------

#clr = ['#F4A460', '#8B7355', '#A52A2A', 'green', 'blue']

def isolate_targets(df, target_names):
    T=[]
    for i in range (0,len(target_names)):
        T.append(df[target_names[i]])
    return(T)
    
T = isolate_targets(df,target_names) 

In [30]:
tar = pd.DataFrame(T)
y = tar.T

In [31]:
# List of pretrained models
models = [sand_model_gbrt, silt_model_gbrt, clay_model_cubist, TOC_model_cubist, CaCO3_model_cubist]  

# List of preprocessed spectra
preprocessed_spectra = [fod_sampled11, fod_sampled11, fod_sampled11, fod_log11, fod_log11] 

In [32]:
# Calculate R2 score
r2_scores = [] 

for i, model in enumerate(models):
    # Predict the soil property values for the preprocessed spectra
    y_pred = model.predict(preprocessed_spectra[i])
    
    # Calculate R2 score
    r2 = find_r2(y_pred, y.iloc[:, i])
    r2_scores.append(r2)

# Print R2 scores for each model
for i, r2 in enumerate(r2_scores):
    print(f"Model {i+1} R2 Score: {r2}")


Model 1 R2 Score: 0.8573567212434977
Model 2 R2 Score: 0.8682389358609224
Model 3 R2 Score: 0.6338895498363157
Model 4 R2 Score: 0.8093619604229948
Model 5 R2 Score: 0.7585512957677691


C:\Users\abhis\AppData\Local\Programs\Python\Python311\Lib\site-packages\cubist\_make_data_string.py:67: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  x = x.applymap(lambda a: a.lstrip())
C:\Users\abhis\AppData\Local\Programs\Python\Python311\Lib\site-packages\cubist\_make_data_string.py:67: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  x = x.applymap(lambda a: a.lstrip())
C:\Users\abhis\AppData\Local\Programs\Python\Python311\Lib\site-packages\cubist\_make_data_string.py:67: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  x = x.applymap(lambda a: a.lstrip())


In [33]:
from tabulate import tabulate

# List of lists containing model names and R2 scores
results = [
    ["sand_model_gbrt", 0.35, 0.857],
    ["silt_model_gbrt", 0.34, 0.868],
    ["clay_model_cubist", 0.43, 0.634],
    ["TOC_model_cubist", 0.5, 0.809],
    ["CaCO3_model_cubist", 0.6, 0.758],
    
]

# Print the results in a table format
print(tabulate(results, headers=['Model', 'R2 (Leave One Out)', 'R2 (Full Prediction)'], tablefmt='pretty'))


+--------------------+--------------------+----------------------+
|       Model        | R2 (Leave One Out) | R2 (Full Prediction) |
+--------------------+--------------------+----------------------+
|  sand_model_gbrt   |        0.35        |        0.857         |
|  silt_model_gbrt   |        0.34        |        0.868         |
| clay_model_cubist  |        0.43        |        0.634         |
|  TOC_model_cubist  |        0.5         |        0.809         |
| CaCO3_model_cubist |        0.6         |        0.758         |
+--------------------+--------------------+----------------------+
